In [3]:
from bs4 import BeautifulSoup as bs
from unidecode import  unidecode
from urllib.parse import unquote
import pandas as pd
import numpy as np
import requests
import glob

In [158]:
def get_players_link(club_link):
    url = f"https://en.wikipedia.org/wiki/{club_link}"
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    players_url_list = []

    for table in soup.find_all('table', class_="wikitable football-squad nogrid"):
        players = table.find_all('span', class_="fn")
        for player in players:
            try:
                players_url_list.append(player.find('a')['href'])
            except:
                pass

    return players_url_list

players_link_list = get_players_link('Arsenal_F.C.')
players_link_list[:4]

['/wiki/Aaron_Ramsdale',
 '/wiki/Kieran_Tierney',
 '/wiki/Ben_White_(footballer)',
 '/wiki/Thomas_Partey']

In [176]:
def get_senior_clubs(player_link):
    url = f"https://en.wikipedia.org{player_link}"
    response = requests.get(url)

    html = response.content
    soup = bs(html, "lxml")

    table = soup.find_all('table', class_="infobox vcard")
    df = pd.read_html(str(table))[0]
    df.columns = range(len(df.columns))

    senior_career_index = df[
            (df[0] == 'Senior career*')
    ].index.values[0] + 1

    header = df.iloc[senior_career_index]

    df_new = df.iloc[
        senior_career_index + 1 :
    ]

    df_new.columns = header

    cpt = 0
    for i in df_new['Apps'].tolist():
        try:
            int(i)
            cpt += 1
        except:
            break

    df_new_cleaned = df_new.head(cpt).reset_index(drop=True)
    df_new_cleaned.columns.name = None
    df_new_cleaned['is_loan'] = df_new_cleaned['Team'].apply(lambda x: 1 if "→ " in x else 0)
    df_new_cleaned['Team'] = (
        df_new_cleaned['Team'].str.replace("→ ", "", regex=True).str.replace(r"\(.*\)", "", regex=True)
    )
    df_new_cleaned.columns = list(map(lambda x: x.replace('(Gls)', 'Goals'), df_new_cleaned.columns.tolist()))
    df_new_cleaned['Goals'] = df_new_cleaned['Goals'].apply(lambda x: x[x.find("(")+1 : x.find(")")])
    df_new_cleaned['Name'] = player_link.replace('/wiki/', '').replace('_', ' ')
    return df_new_cleaned

df = get_senior_clubs('/wiki/Aaron_Ramsdale')
df

,Years,Team,Apps,Goals,is_loan,Name
0,2016–2017,Sheffield United,0,0,0,Aaron Ramsdale
1,2017–2020,AFC Bournemouth,37,0,0,Aaron Ramsdale
2,2018,Chesterfield,19,0,1,Aaron Ramsdale
3,2019,AFC Wimbledon,20,0,1,Aaron Ramsdale
4,2020–2021,Sheffield United,40,0,0,Aaron Ramsdale
5,2021–,Arsenal,46,0,0,Aaron Ramsdale


In [220]:
club = 'Olympique_de_Marseille'
list_df_players_club = []
players_link_list = get_players_link(club)

for player_link in players_link_list:
    print(player_link)
    try:
        list_df_players_club.append(get_senior_clubs(player_link))
    except:
        print('error')
    
df_players_club = pd.concat(list_df_players_club, ignore_index=True)
df_players_club.to_csv(f'players/{club}.csv', index=False)
print(df_players_club.shape)
df_players_club.tail()

/wiki/Simon_Ngapandouetnbu
/wiki/Eric_Bailly
/wiki/Samuel_Gigot
/wiki/Leonardo_Balerdi
/wiki/Matteo_Guendouzi
/wiki/Jonathan_Clauss
/wiki/Gerson_(footballer,_born_1997)
/wiki/Dimitri_Payet
/wiki/Luis_Su%C3%A1rez_(footballer,_born_1997)
/wiki/Bamba_Dieng
/wiki/Pau_L%C3%B3pez
/wiki/Cengiz_%C3%9Cnder
/wiki/Isaak_Tour%C3%A9
/wiki/Valentin_Rongier
/wiki/Pape_Gueye
/wiki/Sead_Kola%C5%A1inac
/wiki/Jordan_Veretout
/wiki/Issa_Kabor%C3%A9
/wiki/Nuno_Tavares
/wiki/Rub%C3%A9n_Blanco
/wiki/Bartu%C4%9F_Elmaz
/wiki/Alexis_S%C3%A1nchez
/wiki/Amine_Harit
/wiki/Chancel_Mbemba
/wiki/Jordan_Amavi
/wiki/Pol_Lirola
/wiki/Oussama_Targhalline
/wiki/Nemanja_Radonji%C4%87
/wiki/Kevin_Strootman
/wiki/Luis_Henrique_(footballer,_born_2001)
/wiki/Konrad_de_la_Fuente
/wiki/Arkadiusz_Milik
/wiki/Bartu%C4%9F_Elmaz
/wiki/Roggerio_Nyakossi
/wiki/Ibtoihi_Hadhari
/wiki/Pedro_Ruiz_(footballer,_born_2000)
/wiki/Salim_Ben_Seghir
(183, 6)


,Years,Team,Apps,Goals,is_loan,Name
178,2021–,Marseille II,0,0,0,"Pedro Ruiz (footballer, born 2000)"
179,2021–2022,NEC,3,0,1,"Pedro Ruiz (footballer, born 2000)"
180,2020–2021,Nice II,9,3,0,Salim Ben Seghir
181,2020–2021,Nice,1,0,0,Salim Ben Seghir
182,2021–,Marseille,0,0,0,Salim Ben Seghir


In [11]:
all_files = glob.glob("players/*.csv")
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True).rename(columns={'Team': 'Teams'})[['Name', 'Teams']]

for c in frame.columns:
    frame[c] = frame[c].apply(lambda x: unidecode(unquote(x.strip()))).str.replace(r"\(.*\)", "", regex=True)

    
frame = frame[
    (~frame['Teams'].str.endswith(' B'))
    & (~frame['Teams'].str.endswith(' II'))
    & (~frame['Teams'].str.endswith(' 2'))
]
frame.drop_duplicates(inplace=True)    
frame.tail()

,Name,Teams
1522,Yaya Soumare,Annecy
1523,Yaya Soumare,Bourg-en-Bresse
1525,Mohamed El Arouch,Lyon
1528,Florent Da Silva,Lyon
1529,Florent Da Silva,Villefranche


In [12]:
pd.set_option('display.max_rows', None)
frame['Teams'].value_counts()

Chelsea                     52
Manchester United           52
Manchester City             47
Arsenal                     46
Monaco                      45
Paris Saint-Germain         42
Barcelona                   40
Marseille                   36
Lyon                        35
Liverpool                   33
Borussia Dortmund           12
Roma                        12
Benfica                     11
Real Madrid                 11
Porto                       10
Aston Villa                  9
Ajax                         9
Inter Milan                  9
Sporting CP                  8
Atletico Madrid              8
Hull City                    8
Schalke 04                   8
Sevilla                      8
Fulham                       7
Bayern Munich                7
Valencia                     7
Sheffield United             7
Southampton                  7
Juventus                     7
Lille                        7
Anderlecht                   6
AC Milan                     6
Reims   

In [13]:
df_final = frame[['Name', 'Teams']].groupby('Name').agg({'Teams': lambda x: list(x)}).reset_index()
df_final['Number_Teams'] = df_final['Teams'].apply(lambda x: len(x))
df_final_workable = df_final[
    (df_final['Number_Teams'] >= 3)
    & (df_final['Name'].apply(lambda x: len(x.split())) < 3)
].copy()
df_final_workable['Name'] = df_final_workable['Name'].apply(lambda x: x.split()[-1])
df_final_workable.tail()

,Name,Teams,Number_Teams
362,Saliba,"[Saint-Etienne, Arsenal, Nice, Marseille]",4
365,Couto,"[Coritiba, Manchester City, Girona, Braga]",4
366,Soumare,"[Lyon, Dijon, Annecy, Bourg-en-Bresse]",4
369,Kone,"[Lille, Reims, Lyon, Elche, Hatayspor, Troyes,...",7
370,Steffen,"[Columbus Crew SC, Pittsburgh Riverhounds, Man...",5


In [14]:
df_final_workable[['Name', 'Teams']].to_json('players.json', orient="records")